## 1. Hidden Markov Model (HMM)

Hidden Markov Model (HMM) 은 길이가 $n$ 인 sequence $x_{1:n} = [x_1, x_2, \dots, x_n]$ 에 대하여 $P(y_{1:n} \vert x_{1:n})$ 가 가장 큰 $y_{1:n}$ 를 찾습니다. 품사 판별의 문제에서는 $n$ 개의 단어로 구성된 단어열에 대하여 각 단어의 품사 $y$ 를 부여하는 것입니다. 이 과정을 HMM 의 decoding 이라 합니다. 그리고 우리가 찾아야 하는 label, $y$ 를 HMM 에서는 state 라 합니다.

이 때 $P(y_{1:n} \vert x_{1:n})$ 는 다음처럼 계산됩니다.

$P(y_{1:n} \vert x_{1:n}) = P(x_1 \vert y_1) \times P(y_1 \vert BOS) \times P(y_2 \vert y_1) \times P(x_2 \vert y_2) \cdots$

위의 식은 현재 시점 $i$ 의 state 인 $y_i$ 를 판별 (classification, or labeling) 하기 위하여 이전 시점의 state 인 $y_{i-1}$ 이 이용됩니다. 이처럼 이전의 한 단계 전의 state 정보를 이용하는 모델을 first-order Markov Model 이라 합니다. 만약 이전 두 단계의 states 정보를 모두 이용한다면 $P(y_i \vert y_{i-2}, y_{i-1})$ 이 학습되어야 하며, 이는 second-order Markov Model 이라 합니다. 그 외의 멀리 떨어진 state 정보는 이용하지 않습니다.

이처럼 state 간의 변화 확률을 transition probability (전이 확률) 라 합니다. HMM 은 각 state 에서 우리가 관측 가능한 값 (observation) 이 발생할 확률이 있다고 가정합니다. 이를 emission probability, $P(x_i \vert y_i)$ 라 합니다. 품사 판별에서는 명사 집합에서 '아이오아이'라는 단어가 존재할 확률 입니다.

숫자 계산에서 곱셈은 덧셈보다 비싼 작업입니다. 그렇기 때문에 확률을 곱하는 작업들은 주로 log 를 씌워 덧셈으로 변환합니다. 위 수식은 아래처럼 변환됩니다.

$log P(y_{1:n} \vert x_{1:n}) = log P(x_1 \vert y_1)+ log P(y_1 \vert S) + log P(y_2 \vert y_1) + log P(x_2 \vert y_2) \cdots$

$log P(x_i \vert y_i)$ 나 $log P(y_i \vert y_{i-1})$ 은 마치 score($x_i, y_i$), score($y_i, y_{i-1}$) 형태와 같습니다.

##  2. Supervised training

학습 말뭉치를 이용하여 품사 판별이나 형태소 분석용 tagger 를 학습하기도 합니다. 학습 말뭉치는 다음과 같은 데이터로 구성되어 있습니다.

아래는 세종 말뭉치의 예시입니다. 세종 말뭉치는 한국어 형태소 분석을 위하여 국립국어원에서 배포한 말뭉치입니다. 한 문장이 (morpheme, tag) 형식으로 기술되어 있습니다. 아래는 네 개의 문장의 스냅샷입니다.

    [['뭐', 'NP'], ['타', 'VV'], ['고', 'EC'], ['가', 'VV'], ['ㅏ', 'EF']]
    [['지하철', 'NNG']]
    [['기차', 'NNG']]
    [['아침', 'NNG'], ['에', 'JKB'], ['몇', 'MM'], ['시', 'NNB'], ['에', 'JKB'], ['타', 'VV'], ['고', 'EC'], ['가', 'VV'], ['는데', 'EF']]

우리는 first-order HMM tagger 를 만들어봅니다. 이는 state 관점에서 bigram 을 이용하기 때문에 bigram HMM tagger 라 불려도 괜찮습니다.

bigram HMM tagger 를 학습하려면 transition probability $P(y_i \vert y_{i-1})$ 와 emittion probability $P(x_i \vert y_i)$ 를 학습해야 합니다. 직관적으로 다음처럼 확률을 정의할 수 있습니다.

- transition prob: $P(y_i \vert y_{i-1}) = \frac{f(y_{i-1}, y_i)}{f(y_{i-1})}$
- emittion prob:   $P(x_i \vert y_i) = \frac{f(x_i, y_i)}{f(y_i)}$

그런데 위의 식이 직관적일 뿐 아니라 Maximum Likelihood Estimation (MLE) 관점에서도 확률을 학습하는 solution 입니다. 즉, 학습 말뭉치에서의 빈도수를 계산하는 것만으로 학습을 할 수 있습니다.


In [1]:
from collections import defaultdict
from pprint import pprint

corpus = [
    [('이것', 'Noun'), ('은', 'Josa'), ('예시', 'Noun'), ('이', 'Adjective'), ('었다', 'Eomi')]
]

우리는 corpus 가 nested list 구조라 가정합니다. 각 문장 sent 는 [(word, tag), (word, tag), ... ] 형식입니다. 우리가 해야 할 일은 각 태그 별로 단어가 발생한 횟수와 $tag_{i-1}, tag_{i}$ 의 횟수를 세는 것 뿐입니다.

문장의 시작을 BOS 로, 끝을 EOS 로 구분하기 위하여 tag sequence 에 BOS 와 EOS 를 추가하여 bigram counting 을 합니다.

In [2]:
BOS = 'BOS'
EOS = 'EOS'

def count(corpus):
    emissions = defaultdict(lambda: defaultdict(int))
    transitions = defaultdict(int)

    for sent in corpus:
        # emission prob
        for word, pos in sent:
            emissions[pos][word] += 1

        words, tags = zip(*sent)
        tags = [BOS] + list(tags) + [EOS]

        # transition prob
        for t0, t1 in zip(tags, tags[1:]):
            transitions[(t0, t1)] += 1

    return {k:dict(v) for k,v in emissions.items()}, dict(transitions)

우리가 계산한 것은 빈도수 입니다. 이를 확률로 변형합니다. Emissions probability 는 각 품사의 빈도수로 (단어, 품사) 발생 횟수를 나눠주면 됩니다. Transitions probability 역시 이전 품사의 빈도수인 base[tag0] 으로 (tag0, tag1) 의 빈도수를 나눠주면 됩니다.

그리고 float 는 곱셈보다 덧셈이 계산이 빠르기 때문에, 미리 math.log 를 이용하여 확률값을 log probability 로 변환하여 줍니다.

In [3]:
import math

def _to_log_prob(emissions, transitions):

    # emission
    base = {tag:sum(words.values()) for tag, words in emissions.items()}
    emissions_ = {tag:{word:math.log(count/base[tag]) for word, count in words.items()}
                  for tag, words in emissions.items()}

    # transition
    base = defaultdict(int)
    for (tag0, tag1), count in transitions.items():
        base[tag0] += count
    transitions_ = {tags:math.log(count/base[tags[0]]) for tags, count in transitions.items()}

    return emissions_, transitions_

Sequential labeling 을 위한 HMM model 의 supervised learning 은 모두 끝났습니다.

## 3. HMM 기반 형태소 분석기 만들기

일단은 클래스로 정리하지 않고, 기능별로 함수를 만들어봅니다.

이를 위해 다음의 데이터를 이용합니다. max word len, min emission, min transition 은 모델이 이용할 parameters 입니다. 미등록 단어에 대한 emission score 와 transition score 입니다. 또한 학습된 사전에서 어느 길이까지 단어를 찾아볼지 정의하기 위하여, 사전에 등록된 단어의 길이 중 가장 긴 길이를 max word len 에 저장합니다.

In [4]:
corpus = [
    [('이것', 'Noun'), ('은', 'Josa'), ('예시', 'Noun'), ('이', 'Adjective'), ('다', 'Eomi')],
    [('이것', 'Noun'), ('도', 'Josa'), ('예시', 'Noun'), ('이', 'Adjective'), ('었다', 'Eomi')],
    [('아이오아이', 'Noun'), ('를', 'Josa'), ('예시', 'Noun'), ('로', 'Josa'), ('이용', 'Noun'), ('하', 'Verb'), ('ㄴ다', 'Eomi')],
    [('공연', 'Noun'), ('이', 'Josa'), ('시작', 'Noun'), ('하', 'Verb'), ('ㄴ다', 'Eomi')],
    [('이것', 'Noun'), ('은', 'Josa'), ('아이', 'Noun'), ('의', 'Josa'), ('예시', 'Noun'), ('이', 'Adjective'), ('다', 'Eomi')],
    [('이', 'Noun'), ('는', 'Josa'), ('숫자', 'Noun'), ('이', 'Adjective'), ('다', 'Eomi')],
    [('예', 'Noun'), ('도', 'Josa'), ('단어', 'Noun'), ('이', 'Adjective'), ('다', 'Eomi')],
]

emissions_freq, transitions_freq = count(corpus)
emissions, transitions = _to_log_prob(emissions_freq, transitions_freq)

In [5]:
max_word_len = max(len(w) for words in emissions.values() for w in words)
min_emission = min(s for words in emissions.values() for s in words.values()) - 0.05
min_transition = min(transitions.values()) - 0.05

print(max_word_len)
print(min_emission)
print(min_transition)

5
-2.822588722239781
-2.1294415416798356


### 3.1. Lookup

앞서 만든 Lemmatizer 를 가져옵니다. 동사, 형용사, 어미 사전은 학습 말뭉치에 있는 단어를 이용하고, lemma_rules 는 실습을 위한 demo set 을 이용합니다.

In [6]:
import sys
sys.path.append('./korean_lemmatizer/')
from soylemma import Lemmatizer

rules = {
    '한': {('하', 'ㄴ')},
    '였': {('이', '었')}
}

lemmatizer = Lemmatizer(
    verbs = emissions.get('Verb', {}),
    adjectives = emissions.get('Adjective', {}),
    eomis = emissions.get('Eomi', {}),
    lemma_rules = rules
)

lemmatizer.analyze('한다')

[(('하', 'Verb'), ('ㄴ다', 'Eomi'))]

In [7]:
print(lemmatizer.verbs)
print(lemmatizer.adjectives)
print(lemmatizer.eomis)
print(lemmatizer.lemma_rules)

{'하'}
{'이'}
{'다', '었다', 'ㄴ다'}
{'한': {('하', 'ㄴ')}, '였': {('이', '었')}}


문장이 입력되면 띄어쓰기 기준으로 이를 분리합니다. 어절 단위로 우리가 알고 있는 단어들이 존재하는지 확인합니다. offset 은 현재까지 탐색한 문장에서의 글자수 입니다. 문장에서의 다음 어절의 시작점을 알려줍니다.

In [8]:
def sentence_lookup(sentence, emissions, lemmatizer, max_word_len):
    sent = []
    for eojeol in sentence.split():
        sent += eojeol_lookup(
            eojeol,
            emissions,
            lemmatizer,
            max_word_len,
            offset = len(sent)
        )
    return sent

단어를 (morph, tag, morph, tag, begin, end) 형식으로 저장합니다. 이는 용언처럼 형태가 변하는 (활용, conjugation) 단어의 정보를 손쉽게 저장하기 위함입니다. 이를 namedtuple 로 만들어 가독성을 좋게 합니다.

또한 용언이 아니라 하더라도 tag1 에 tag0 을 함께 입력합니다. 이는 이후에 그래프를 만드는데 도움이 됩니다.

In [9]:
from collections import namedtuple

class Word(namedtuple('Word', 'morph0 tag0 morph1 tag1 begin end is_compound')):    
    def __repr__(self):
        if not self.is_compound:
            return 'Word({}/{}, {}, {})'.format(
                self.morph0, self.tag0, self.begin, self.end)
        else:
            return 'Word({}/{} + {}/{}, {}, {})'.format(
                self.morph0, self.tag0, self.morph1, self.tag1, self.begin, self.end)

word0 = Word('시작', 'Noun', None, 'Noun', 1, 3, False)
word1 = Word('하', 'Verb', 'ㄴ다', 'Eomi', 3, 5, True)

print(word0)
print(word1)

Word(시작/Noun, 1, 3)
Word(하/Verb + ㄴ다/Eomi, 3, 5)


어절에서의 단어 시작점 b 로부터 사전에 등록된 단어의 최대 길이 max_word_len 까지 확장하며 substring 을 surface 에 잘라둡니다. 이 substring 이 사전에 존재하는지 get_pos 함수를 이용하여 확인합니다. 품사가 존재한다면 이는 우리가 학습한 단어라는 의미입니다.

get_pos 함수에서 동사, 형용사, 어미는 string match 를 하지 않습니다.  이들은 lemmatizer 가 확인할 것이기 때문입니다.

In [10]:
def eojeol_lookup(eojeol, emissions, lemmatizer, max_word_len, offset=0):
    n = len(eojeol)
    pos = [[] for _ in range(n)]
    for b in range(n):
        for r in range(1, max_word_len+1):
            e = b+r
            if e > n:
                continue
            sub = eojeol[b:e]

            # string match
            for tag in get_pos(sub, emissions, max_word_len):
                pos[b].append(Word(sub, tag, None, tag, b + offset, e + offset, False))

            # lemmatizer
            for (w0, t0), (w1, t1) in lemmatizer.analyze(sub):
                pos[b].append(Word(w0, t0, w1, t1, b + offset, e + offset, True))
    return pos

def get_pos(sub, emissions, max_word_len):
    tags = []
    for tag, words in emissions.items():
        if tag == 'Verb' or tag == 'Adjective' or tag == 'Eomi':
            continue
        if sub in words:
            tags.append(tag)
    return tags

In [11]:
eojeol_lookup('시작한다', emissions, lemmatizer, max_word_len)

[[Word(시작/Noun, 0, 2)], [], [Word(하/Verb + ㄴ다/Eomi, 2, 4)], []]

eojeol_lookup 함수를 이용하면 어절의 길이만큼의 nested list 가 return 됩니다. list 의 각 위치는 어절에서의 글자의 시작지점입니다.

In [12]:
eojeol_lookup('예시이다', emissions, lemmatizer, max_word_len)

[[Word(예/Noun, 0, 1), Word(예시/Noun, 0, 2)],
 [],
 [Word(이/Noun, 2, 3), Word(이/Josa, 2, 3), Word(이/Adjective + 다/Eomi, 2, 4)],
 []]

문장에 대한 lookup 을 수행한 결과 입니다. '예시'는 문장에서 네번째로 등장하는 단어이기 때문에 begin index 가 3 임을 확인할 수 있습니다.

In [13]:
sentence_lookup('이것은 예시이다', emissions, lemmatizer, max_word_len)

[[Word(이/Noun, 0, 1), Word(이/Josa, 0, 1), Word(이것/Noun, 0, 2)],
 [],
 [Word(은/Josa, 2, 3)],
 [Word(예/Noun, 3, 4), Word(예시/Noun, 3, 5)],
 [],
 [Word(이/Noun, 5, 6), Word(이/Josa, 5, 6), Word(이/Adjective + 다/Eomi, 5, 7)],
 []]

### 3.2. 그래프 만들기

앞서 어절과 문장에서 알려진 단어를 찾는 과정을 구현하였습니다. 이 단어들 중에서 앞의 단어의 end index 와 뒤의 단어의 begin index 가 같은 경우, 이 둘을 연결할 수 있습니다.

이는 다음의 과정으로 구현할 수 있습니다.

    links = []
    for words in sent[:-1]:
        for word in words:
            for adjacent in sent[word.end]:
                links.append((word, adjacent))

그러나 한 단어의 end index 에서부터 시작하는 단어가 없을 경우에는 링크가 만들어지지 않습니다. 이 때에는 가능한 가장 가까운 다음 단어의 begin index 를 찾아야 합니다. 이를 위하여 다음의 함수를 구현합니다. offset 이후의 지점에서 sent[i] 가 empty 가 아닌 가장 빠른 지점을 return 합니다.

    def get_nonempty_first(sent, end, offset=0):
        for i in range(offset, end):
            if sent[i]:
                return i
        return offset

만약 문장의 끝부분까지 아는 단어가 존재하지 않을수도 있습니다. 이를 방지하기 위해 문장의 끝을 표시하는 eos 를 sent 에 추가합니다.

    sent = sentence_lookup(sentence)
    n_char = len(sent) + 1

    eos = Word('EOS', 'EOS', 'EOS', 'EOS', n_char-1, n_char, False)
    sent.append([eos])

그리고 end index 로부터 시작하는 단어가 없을 경우, 가장 가까운 단어의 시작지점까지 부분어절을 잘라 'Unk' 태그를 부여합니다. 

    links = []
    for words in sent[:-1]:
        for word in words:
            if not sent[word.end]:
                b = get_nonempty_first(sent, n_char, word.end)
                unk = Word(chars[end:b], 'Unk', 'Unk', word.end, b)
                links.append((word, unk))
            for adjacent in sent[word.end]:
                links.append((word, adjacent))

하지만 모르는 단어로부터 아는 단어까지의 링크가 아직 만들어지지 않았습니다. 품사가 'Unk' 인 단어들을 찾아 그 단어의 end index 로부터 시작하는 단어들과 링크를 만듭니다.

    unks = {to_node for _, to_node in links if to_node.tag0 == 'Unk'}
    for unk in unks:
        for adjacent in sent[unk.end]:
            links.append((unk, adjacent))

마지막으로 그래프의 시작점에 bos 를 추가합니다. 문장의 맨 앞에 있는 단어들과 bos 간에 링크를 추가합니다.

    bos = Word('BOS', 'BOS', 'BOS', 'BOS', 0, 0, False)
    for word in sent[0]:
        links.append((bos, word))

Shortest path 에서 bos 와 eos 는 시작 마디와 끝 마디로 이용됩니다. 이 값을 함께 return 합니다.

In [14]:
def generate_link(sentence, emissions, lemmatizer, max_word_len):

    def get_nonempty_first(sent, end, offset=0):
        for i in range(offset, end):
            if sent[i]:
                return i
        return offset

    chars = sentence.replace(' ','')
    sent = sentence_lookup(
        sentence,
        emissions,
        lemmatizer,
        max_word_len
    )
    n_char = len(sent) + 1

    eos = Word('EOS', 'EOS', 'EOS', 'EOS', n_char-1, n_char, False)
    sent.append([eos])

    i = get_nonempty_first(sent, n_char)
    if i > 0:
        sent[0].append(Word(chars[:i], 'Unk', None, 'Unk', 0, i, False))

    links = []
    for words in sent[:-1]:
        for word in words:
            if not sent[word.end]:
                b = get_nonempty_first(sent, n_char, word.end)
                unk = Word(chars[word.end:b], 'Unk', None, 'Unk', word.end, b, False)
                links.append((word, unk))
            for adjacent in sent[word.end]:
                links.append((word, adjacent))

    unks = {to_node for _, to_node in links if to_node.tag0 == 'Unk'}
    for unk in unks:
        for adjacent in sent[unk.end]:
            links.append((unk, adjacent))

    bos = Word('BOS', 'BOS', 'BOS', 'BOS', 0, 0, False)
    for word in sent[0]:
        links.append((bos, word))
    links = sorted(links, key=lambda x:(x[0].begin, x[1].end))

    return links, bos, eos

이 함수를 적용하여 만든 링크입니다. '것'이라는 단어는 알려지지 않았기 때문에 'Unk' 품사로 인식되었습니다. 또한 '였다'는 '이/Adjective + 었다/Eomi' 로 인식되어 하나의 마디를 이루고 있습니다.

In [15]:
links, bos, eos = generate_link(
    '이것은 예시였다',
    emissions,
    lemmatizer,
    max_word_len
)

pprint(links)

[(Word(BOS/BOS, 0, 0), Word(이/Noun, 0, 1)),
 (Word(BOS/BOS, 0, 0), Word(이/Josa, 0, 1)),
 (Word(이/Noun, 0, 1), Word(것/Unk, 1, 2)),
 (Word(이/Josa, 0, 1), Word(것/Unk, 1, 2)),
 (Word(BOS/BOS, 0, 0), Word(이것/Noun, 0, 2)),
 (Word(이것/Noun, 0, 2), Word(은/Josa, 2, 3)),
 (Word(것/Unk, 1, 2), Word(은/Josa, 2, 3)),
 (Word(은/Josa, 2, 3), Word(예/Noun, 3, 4)),
 (Word(은/Josa, 2, 3), Word(예시/Noun, 3, 5)),
 (Word(예/Noun, 3, 4), Word(시/Unk, 4, 5)),
 (Word(예시/Noun, 3, 5), Word(이/Adjective + 었다/Eomi, 5, 7)),
 (Word(시/Unk, 4, 5), Word(이/Adjective + 었다/Eomi, 5, 7)),
 (Word(이/Adjective + 었다/Eomi, 5, 7), Word(EOS/EOS, 7, 8))]


만든 링크에 가중치를 부여합니다. HMM 의 모델을 그래프로 표현하기 위해서는 현재 마디로 유입되는 앞 마디의 단어로부터 지금 단어로 이동하는 transition probability 와 현재 마디의 단어, 품사가 발생할 emission probability 의 곱, 혹은 두 확률의 log 값의 합을 가중치로 이용하면 됩니다. HMM 학습 시 보지 못했던 단어와 transition 일 수 있기 때문에 min_emission 과 min_transition 을 이용하여 Key Error 를 방지합니다.

    w = emissions.get(to_node.tag0, {}).get(to_node.morph0, min_emission)
    w += transitions.get((from_node.tag1, to_node.tag0), min_transition)

단, '였다 = 이/Adjective + 었다/Eomi' 의 경우, 하나의 마디에 두 개의 형태소가 포함된 구조이므
로, 이 때에는 마디 안에서의 transition 도 고려해야 합니다. tuple 로 표현된 마디의 첫번째 요소를 ' + '로 나눕니다. 이 길이가 2 라면 이는 '이 + 었다'처럼 두 형태소가 하나의 마디를 이루는 경우입니다.

    if to_node.is_compound:
        w += emissions.get(to_node.tag1, {}).get(to_node.morph0, min_emission)
        w += transitions.get(to_node.tag1, {}).get(to_node.tag0, min_transition)

그리고 이 과정에서 다른 품사보다 명사를 더 선호한다거나, 길이가 짧은 단어에 페널티를 부여할 수도 있습니다.

In [16]:
def add_weight(links, emissions, transitions, min_emission, min_transition):

    def weight(from_node, to_node):

        # score of first word
        w = emissions.get(to_node.tag0, {}).get(to_node.morph0, min_emission)
        w += transitions.get((from_node.tag1, to_node.tag0), min_transition)

        # penalty example
        if to_node.tag0 == 'Noun' and len(to_node.morph0) == 1:
            w = w / 2

        # score of second word
        if to_node.is_compound:
            w += emissions.get(to_node.tag1, {}).get(to_node.morph1, min_emission)
            w += transitions.get(to_node.tag1, {}).get(to_node.tag0, min_transition)
        return w

    graph = []
    for from_node, to_node in links:
        edge = (from_node, to_node, weight(from_node, to_node))
        graph.append(edge)
    return graph

링크에 가중치를 더한 결과입니다. 미등록 단어인 '것'과 연결된 경우에는 가장 작은 가중치가 부여되었습니다.

In [17]:
graph = add_weight(links, emissions, transitions, min_emission, min_transition)

for node_0, node_1, weight in graph:
    print('{}    \t->    {}: {:.4}'.format(node_0, node_1, weight))

Word(BOS/BOS, 0, 0)    	->    Word(이/Noun, 0, 1): -1.386
Word(BOS/BOS, 0, 0)    	->    Word(이/Josa, 0, 1): -4.327
Word(이/Noun, 0, 1)    	->    Word(것/Unk, 1, 2): -4.952
Word(이/Josa, 0, 1)    	->    Word(것/Unk, 1, 2): -4.952
Word(BOS/BOS, 0, 0)    	->    Word(이것/Noun, 0, 2): -1.674
Word(이것/Noun, 0, 2)    	->    Word(은/Josa, 2, 3): -2.079
Word(것/Unk, 1, 2)    	->    Word(은/Josa, 2, 3): -3.634
Word(은/Josa, 2, 3)    	->    Word(예/Noun, 3, 4): -1.386
Word(은/Josa, 2, 3)    	->    Word(예시/Noun, 3, 5): -1.386
Word(예/Noun, 3, 4)    	->    Word(시/Unk, 4, 5): -4.952
Word(예시/Noun, 3, 5)    	->    Word(이/Adjective + 었다/Eomi, 5, 7): -5.239
Word(시/Unk, 4, 5)    	->    Word(이/Adjective + 었다/Eomi, 5, 7): -6.205
Word(이/Adjective + 었다/Eomi, 5, 7)    	->    Word(EOS/EOS, 7, 8): -2.823


### 3.3. 포드 알고리즘을 이용한 최단경로 찾기

최단경로를 찾는 포드 알고리즘은는 d[u] + w(u,v) < d[v] 이면 d[v] = d[u] + w(u,v) 로 대체합니다. 이를 d[u] + w(u,v) > d[v] 이면 d[v] = d[u] + w(u,v) 이 되도록 변경하면 최장경로를 구하는 함수로 바꿀 수 있습니다. HMM 은 log 확률 합의 최대값을 지니는 sequence 를 찾습니다. 이를 위하여 아래의 ford_list 함수는 최장경로를 찾도록 식을 변형하였습니다.

In [18]:
from shortestpath import ford_list

nodes = {node for edge in graph for node in edge[:2]}

# choose optimal sequence
path, cost = ford_list(graph, nodes, bos, eos)

앞서 생성한 그래프를 ford_list 함수에 입력하면 품사 판별이 된 문장이 선택됩니다.

In [19]:
pprint(path)

[Word(BOS/BOS, 0, 0),
 Word(이것/Noun, 0, 2),
 Word(은/Josa, 2, 3),
 Word(예시/Noun, 3, 5),
 Word(이/Adjective + 었다/Eomi, 5, 7),
 Word(EOS/EOS, 7, 8)]


### 3.4. 형태소 형식으로 만들기

위 결과에서 '이 + 었다'를 각각 하나의 형태소로 분해하면 형태소 분석의 결과가 됩니다. 반대로 '이 + 었다' 를 '였다/Adjective' 로 인식하면 품사 판별이 됩니다. 형태소 분석 결과로 만들기 위해서 ' + ' 를 기준으로 단어를 나눕니다.

In [20]:
def flatten(path):
    pos = []
    for word in path:        
        pos.append((word.morph0, word.tag0))
        if word.is_compound:
            pos.append((word.morph1, word.tag1))
    return pos

pos = flatten(path)
pprint(pos)

[('BOS', 'BOS'),
 ('이것', 'Noun'),
 ('은', 'Josa'),
 ('예시', 'Noun'),
 ('이', 'Adjective'),
 ('었다', 'Eomi'),
 ('EOS', 'EOS')]


이번에는 미등록 단어 'tt' 가 포함된 문장을 넣어 형태소 분석을 한 결과까지 한 번에 얻어봅니다. 'tt' 가 미등록 단어로 인식되었습니다.

In [21]:
links, bos, eos = generate_link('tt도예시였다', emissions, lemmatizer, max_word_len)
graph = add_weight(links, emissions, transitions, min_emission, min_transition)
nodes = {node for edge in graph for node in edge[:2]}
path, cost = ford_list(graph, nodes, bos, eos)
pos = flatten(path)
pprint(pos)

[('BOS', 'BOS'),
 ('tt', 'Unk'),
 ('도', 'Josa'),
 ('예시', 'Noun'),
 ('이', 'Adjective'),
 ('었다', 'Eomi'),
 ('EOS', 'EOS')]


### 3.5. 미등록단어의 품사 추정

이번에는 tt 의 품사를 추정하는 함수를 만듭니다. 'tt' 는 문장의 맨 앞에 시작했으며 뒤에 '-도/Josa'가 위치하기 때문에 명사일 가능성이 높습니다. 이를 HMM 의 식으로 표현하면 앞 단어의 품사에서 tt 가 가질 수 있는 품사로의 transtion probability 와 tt 가 가질 수 있는 품사에서 다음 단어로의 transition probability 의 곱이 가장 큰 품사를 찾으면 'Noun' 이라는 의미입니다.

    for i, pos_i in enumerate(pos[:-1]):

        ...
        # previous -> current transition
        if i == 1:
            tag_prob = {tag:prob for tag, prob in begin.items()}
        else:
            tag_prob = {
                tag:prob for (prev_tag, tag), prob in transition.items()
                if prev_tag == pos[i-1][1]
            }

        # current -> next transition
        for (tag, next_tag), prob in transition.items():
            if next_tag == pos[i+1][1]:
                tag_prob[tag] = tag_prob.get(tag, 0) + prob

만약 앞, 뒤 단어의 품사들조차 이용할 수 없는 상황이라면 이를 명사로 추정합니다. 한국어 단어 중 명사가 가장 많은 단어를 보유하고 있기 때문에 이러한 추정은 자연스럽습니다.

    for i, pos_i in enumerate(pos[:-1]):
        if not tag_prob:
            infered_tag = 'Noun'
        else:
            infered_tag = sorted(tag_prob, key=lambda x:-tag_prob[x])[0]

In [22]:
def inference_unknown(pos, transitions):
    pos_ = []
    for i, pos_i in enumerate(pos[:-1]):
        if not (pos_i[1] == 'Unk'):
            pos_.append(pos_i)
            continue

        # previous -> current transition
        tag_prob = {
            tag:prob for (prev_tag, tag), prob in transitions.items()
            if prev_tag == pos[i-1][1]
        }

        # current -> next transition
        for (tag, next_tag), prob in transitions.items():
            if next_tag == pos[i+1][1]:
                tag_prob[tag] = tag_prob.get(tag, 0) + prob

        # prevent BOS, EOS
        tag_prob = {tag:prob for tag, prob in tag_prob.items() if tag != 'BOS' and tag != 'EOS'}

        if not tag_prob:
            infered_tag = 'Noun'
        else:
            infered_tag = sorted(tag_prob, key=lambda x:-tag_prob[x])[0]
        pos_.append((pos_i[0], infered_tag))

    return pos_ + pos[-1:]

추정 함수를 거친 결과 'tt' 는 명사로 인식됩니다.

In [23]:
pos_ = inference_unknown(pos, transitions)
pprint(pos_)

[('BOS', 'BOS'),
 ('tt', 'Noun'),
 ('도', 'Josa'),
 ('예시', 'Noun'),
 ('이', 'Adjective'),
 ('었다', 'Eomi'),
 ('EOS', 'EOS')]


### 3.6. 후처리

위 결과의 앞/뒤에 존재하는 BOS 와 EOS 를 제거하여 형태소 분석 결과를 정리합니다.

In [24]:
def postprocessing(pos):
    return pos[1:-1]

postprocessing(pos_)

[('tt', 'Noun'),
 ('도', 'Josa'),
 ('예시', 'Noun'),
 ('이', 'Adjective'),
 ('었다', 'Eomi')]

### 3.7. 사용자 사전 추가

HMM 기반 모델은 사용자 사전을 추가하기가 쉽습니다. 추가할 (단어, 품사)를 emission probability 에 추가만 하여도 되기 때문입니다.

혹은 존재하는 단어라 하더라도 사용자가 원하는 선호도를 score 로 업데이트 할 수도 있습니다.

In [25]:
def add_user_dictionary(word, tag, score, emissions):
    if not (tag in emission):
        emissions[tag] = {word: score}
    else:
        emissions[tag][word] = score
    return emissions

### 3.8. 클래스 화 하기

위 과정들을 클래스로 정리합니다.

In [26]:
from soylemma import Lemmatizer
from shortestpath import ford_list

class HMMTagger:
    def __init__(self, emissions, transitions, lemmatize):
        self.emissions = emissions
        self.transitions = transitions
        self.lemmatize = lemmatize

        self._max_word_len = max(
            len(w) for words in emissions.values() for w in words)
        self._min_emission = min(
            s for words in emissions.values() for s in words.values()) - 0.05
        self._min_transition = min(transitions.values()) - 0.05

    def tag(self, sentence):
        # lookup & generate graph
        links, bos, eos = self._generate_link(sentence)
        graph = self._add_weight(links)

        # find optimal path
        nodes = {node for edge in graph for node in edge[:2]}
        path, cost = ford_list(graph, nodes, bos, eos)
        pos = self._flatten(path)

        # infering tag of unknown words
        pos = self._inference_unknown(pos)

        # post processing
        pos = self._postprocessing(pos)

        return pos

    def _sentence_lookup(self, sentence):
        sent = []
        for eojeol in sentence.split():
            sent += self._eojeol_lookup(eojeol, offset=len(sent))
        return sent

    def _eojeol_lookup(self, eojeol, offset=0):
        n = len(eojeol)
        pos = [[] for _ in range(n)]
        for b in range(n):
            for r in range(1, self._max_word_len+1):
                e = b+r
                if e > n:
                    continue
                sub = eojeol[b:e]

                # string match
                for tag in self._get_pos(sub):
                    pos[b].append(Word(sub, tag, None, tag, b + offset, e + offset, False))

                # lemmatizer
                for (w0, t0), (w1, t1) in self.lemmatize(sub):
                    pos[b].append(Word(w0, t0, w1, t1, b + offset, e + offset, True))
        return pos

    def _get_pos(self, sub):
        tags = []
        for tag, words in self.emissions.items():
            if tag == 'Verb' or tag == 'Adjective' or tag == 'Eomi':
                continue
            if sub in words:
                tags.append(tag)
        return tags

    def _generate_link(self, sentence):

        def get_nonempty_first(sent, end, offset=0):
            for i in range(offset, end):
                if sent[i]:
                    return i
            return offset

        chars = sentence.replace(' ','')
        sent = self._sentence_lookup(sentence)
        n_char = len(sent) + 1

        eos = Word('EOS', 'EOS', None, 'EOS', n_char-1, n_char, False)
        sent.append([eos])

        i = get_nonempty_first(sent, n_char)
        if i > 0:
            sent[0].append(Word(chars[:i], 'Unk', None, 'Unk', 0, i, False))

        links = []
        for words in sent[:-1]:
            for word in words:
                if not sent[word.end]:
                    b = get_nonempty_first(sent, n_char, word.end)
                    unk = Word(chars[word.end:b], 'Unk', None, 'Unk', word.end, b, False)
                    links.append((word, unk))
                for adjacent in sent[word.end]:
                    links.append((word, adjacent))

        unks = {to_node for _, to_node in links if to_node[1] == 'Unk'}
        for unk in unks:
            for adjacent in sent[unk.end]:
                links.append((unk, adjacent))

        bos = Word('BOS', 'BOS', None, 'BOS', 0, 0, False)
        for word in sent[0]:
            links.append((bos, word))
        links = sorted(links, key=lambda x:(x[0][3], x[1][4]))

        return links, bos, eos

    def _add_weight(self, links):

        def weight(from_node, to_node):

            # score of first word
            w = self.emissions.get(to_node.tag0, {}).get(to_node.morph0, self._min_emission)
            w += self.transitions.get((from_node.tag1, to_node.tag0), self._min_transition)

            # score of second word
            if to_node.is_compound:
                w += self.emissions.get(to_node.tag1, {}).get(to_node.morph1, self._min_emission)
                w += self.transitions.get(to_node.tag0, {}).get(to_node.tag1, self._min_transition)
            return w

        graph = []
        for from_node, to_node in links:
            edge = (from_node, to_node, weight(from_node, to_node))
            graph.append(edge)
        return graph

    def _flatten(self, path):
        pos = []
        for word in path:
            pos.append((word.morph0, word.tag0))
            if word.is_compound:
                pos.append((word.morph1, word.tag1))
        return pos

    def _inference_unknown(self, pos):
        pos_ = []
        for i, pos_i in enumerate(pos[:-1]):
            if not (pos_i[1] == 'Unk'):
                pos_.append(pos_i)
                continue

            # previous -> current transition
            tag_prob = {
                tag:prob for (prev_tag, tag), prob in self.transitions.items()
                if prev_tag == pos[i-1][1]
            }

            # current -> next transition
            for (tag, next_tag), prob in self.transitions.items():
                if next_tag == pos[i+1][1]:
                    tag_prob[tag] = tag_prob.get(tag, 0) + prob

            # prevent BOS, EOS
            tag_prob = {tag:prob for tag, prob in tag_prob.items() if tag != 'BOS' and tag != 'EOS'}

            if not tag_prob:
                infered_tag = 'Noun'
            else:
                infered_tag = sorted(tag_prob, key=lambda x:-tag_prob[x])[0]
            pos_.append((pos_i[0], infered_tag))

        return pos_ + pos[-1:]

    def _postprocessing(self, pos):
        return pos[1:-1]

    def add_user_dictionary(self, word, tag, score):
        if not (tag in self.emissions):
            self.emissions[tag] = {word: score}
        else:
            self.emissions[tag][word] = score

정리한 품사 판별기를 이용하여 앞의 예시 문장의 형태소 분석 결과를 다시 확인합니다.

In [27]:
hmm_tagger = HMMTagger(emissions, transitions, lemmatizer.analyze)
hmm_tagger.tag('tt도예시였다')

[('tt', 'Noun'),
 ('도', 'Josa'),
 ('예시', 'Noun'),
 ('이', 'Adjective'),
 ('었다', 'Eomi')]

In [28]:
hmm_tagger.tag('공연은단어의예시였다')

[('공연', 'Noun'),
 ('은', 'Josa'),
 ('단어', 'Noun'),
 ('의', 'Josa'),
 ('예시', 'Noun'),
 ('이', 'Adjective'),
 ('었다', 'Eomi')]

In [29]:
hmm_tagger.tag('아이오아이와공연은단어의예시였다tt는이것을이용한다')

[('아이오아이', 'Noun'),
 ('와', 'Josa'),
 ('공연', 'Noun'),
 ('은', 'Josa'),
 ('단어', 'Noun'),
 ('의', 'Josa'),
 ('예시', 'Noun'),
 ('이', 'Adjective'),
 ('었다', 'Eomi'),
 ('tt', 'Noun'),
 ('는', 'Josa'),
 ('이것', 'Noun'),
 ('을', 'Josa'),
 ('이용', 'Noun'),
 ('하', 'Verb'),
 ('ㄴ다', 'Eomi')]